In [ ]:
# 预测涨幅测评（标记3） - 加了 few-shot 示例支持

import os
import json
import time
import random
from http import HTTPStatus
from dashscope import Application

# 配置 API Key 和 App ID, QWEN-TURBO
api_key = '##'
app_id = '##'



# ========== Few-shot 示例配置 ==========
"""
#选择
shot_examples = [
    {'id': '600252', 'index': 101},
    {'id': '600255', 'index': 102},
    {'id': '600256', 'index': 103}
]


#填空
shot_examples = [
    {'id': '601318', 'index': 3001},
    {'id': '601369', 'index': 3002},
    {'id': '601996', 'index': 3003}
]

"""

#判断
shot_examples = [
    {'id': '002300', 'index': 4048},
    {'id': '002301', 'index': 4049},
    {'id': '600335', 'index': 4050}

]

# 文件路径
qa_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\predict_allQA2.json'
data_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\predict_data2.json'
output_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\expresult\predictfs26.json'

# 读取问答对
with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data_full = json.load(f)

# 读取股价数据为 dict
with open(data_path, 'r', encoding='utf-8-sig') as f:
    stock_data_list = json.load(f)
    stock_data = {
        item['id']: item['data']
        for item in stock_data_list
        if 'id' in item and 'data' in item
    }

# 格式化股价历史数据
def format_stock_data(data_list):
    return '\n'.join([
        f"{d['Date']}: 开盘={float(d['Open']):.2f}, 收盘={float(d['Close']):.2f}, 最高={float(d['High']):.2f}, "
        f"最低={float(d['Low']):.2f}, 成交量={d.get('Volume', '0')}, 涨跌幅={d.get('Change', '0')}%"
        for d in data_list
    ])

# 准备 few-shot 示例数据
example_data = []
for ex in shot_examples:
    idx, stock_id = ex['index'], ex['id']
    if idx < 0 or idx >= len(qa_data_full):
        raise IndexError(f"示例索引 {idx} 超出范围")
    qa_item = qa_data_full[idx]
    if qa_item['id'] != stock_id:
        raise ValueError(f"索引 {idx} 样本 ID 不匹配：{qa_item['id']} vs {stock_id}")
    example_data.append(qa_item)

print("\n📋 Few-shot 示例配置：")
for ex in example_data:
    print(f"📘 示例问题: {ex['question']} - ID: {ex['id']}")

# 取样（可根据需要修改）
qa_data = qa_data_full[3619:3719]

results = []
MAX_RETRIES = 3
RETRY_DELAY = 5

# 判断是否正确
def is_correct(gt, pred, qtype):
    gt = gt.strip()
    pred = pred.strip()
    if qtype == "填空题":
        return [x.strip() for x in gt.split(',')] == [x.strip() for x in pred.split(',')]
    return gt == pred

# 计算误差（适用于填空题）
def calc_errors(gt, pred):
    try:
        gt_vals = [float(x.strip()) for x in gt.split(',')]
        pred_vals = [float(x.strip()) for x in pred.split(',')]
        if len(gt_vals) != len(pred_vals):
            return None, None, None
        errors = [abs(p - g) for p, g in zip(pred_vals, gt_vals)]
        mape_vals = [abs((p - g) / g) if g != 0 else 0 for p, g in zip(pred_vals, gt_vals)]
        rbias_vals = [(p - g) / g if g != 0 else 0 for p, g in zip(pred_vals, gt_vals)]
        return errors, mape_vals, rbias_vals
    except:
        return None, None, None

all_mape = []
all_rbias = []

# 主循环
for item in qa_data:
    stock_id = item['id']
    qtype = item['type']
    question = item['question']
    answer = item['answer']
    date = item.get('date', '')

    raw_data = stock_data.get(stock_id, [])
    if not raw_data:
        print(f"⚠️ {stock_id} 无历史数据")
        continue

    stock_history = format_stock_data(raw_data)

    if 'choices' in item:
        full_question = question.strip() + '\n' + '\n'.join(item['choices'])
    else:
        full_question = question.strip()

    # 构造 few-shot 示例字符串
    examples_str = ''
    for ex in example_data:
        ex_history = format_stock_data(stock_data.get(ex['id'], []))
        if not ex_history:
            continue
        if 'choices' in ex:
            ex_question = ex['question'].strip() + '\n' + '\n'.join(ex['choices'])
        else:
            ex_question = ex['question'].strip()
        examples_str += f"股票历史数据：\n{ex_history}\n问题：{ex_question}\n答案：{ex['answer']}\n\n"

    print(f"\n🟡 正在处理股票ID: {stock_id}")
    print(f"📋 问题:\n{full_question}")

    # 构造 prompt
    prompt = f
    
    """
"You are a financial expert. Please answer the corresponding questions based on the stock historical price data provided below. Note,Please answer strictly according to the standard:

(1) Give the answer directly, without explanation;
(2) For fill-in-the-blank questions, strictly fill in all blanks in order, using Chinese punctuation and format;
If the answer involves a percentage, do not include the percent sign; for large numerical values, do not use commas; strictly follow the decimal point requirements indicated in the question, and add 0s if necessary;
(3) For true/false questions, simply answer 'Correct' or 'Incorrect';
(4) For multiple-choice questions, choose only one from A/B/C/D.

# Below are some examples:
{examples_str}
Stock historical data is as follows:
{stock_history}
The questions are as follows:
{question}"

"""

    # 调用模型
    for attempt in range(MAX_RETRIES):
        try:
            response = Application.call(
                api_key=api_key,
                app_id=app_id,
                prompt=prompt
            )

            if response.status_code == HTTPStatus.OK:
                pred = response.output.text.strip()
                correct = is_correct(answer, pred, qtype)

                result = {
                    "id": stock_id,
                    "date": date,
                    "type": qtype,
                    "question": full_question,
                    "ground_truth": answer,
                    "model_answer": pred,
                    "correct": correct
                }

                if qtype == "填空题":
                    errors, mape_vals, rbias_vals = calc_errors(answer, pred)
                    if errors is not None:
                        result["errors"] = errors
                        result["MAPE"] = mape_vals
                        result["Rbias"] = rbias_vals
                        all_mape.extend(mape_vals)
                        all_rbias.extend(rbias_vals)

                results.append(result)
                break
            else:
                print(f"{stock_id} 请求失败: {response.status_code}")
                break
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"{stock_id} 错误重试: {str(e)}")
                time.sleep(RETRY_DELAY + random.uniform(0, 2))
            else:
                print(f"{stock_id} 错误跳过")

# 保存结果
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# 输出评估指标
total = len(results)
correct_count = sum(1 for r in results if r['correct'])
accuracy = correct_count / total if total else 0

mape = sum(all_mape) / len(all_mape) if all_mape else 0
rbias = sum(all_rbias) / len(all_rbias) if all_rbias else 0

print(f"\n✅ 共处理: {total} 条")
print(f"🎯 正确: {correct_count} 条，准确率: {accuracy:.2%}")
print(f"📉 平均绝对百分比误差 (MAPE): {mape:.4%}")
print(f"📈 平均相对偏差 (Rbias): {rbias:.4%}")


In [ ]:
# 更新重试
# trend趋势分析，文本填空用加上反义词典的Bertscore,数值填空现在是使用准确率+MAPE+RBias
# 标记4 + few-shot 示例支持

import os
import json
import time
import random
import re
from http import HTTPStatus
from dashscope import Application
from bert_score import score as bert_score

# 配置 API Key 和 App ID
api_key = '##'
app_id = '##'

# ========== Few-shot 示例配置 ==========

"""
#判断
shot_examples = [
    {'id': '603212', 'index': 4001},
    {'id': '603213', 'index': 4002},
    {'id': '603214', 'index': 4003}
]
"""
#填空
shot_examples = [
    {'id': '000518', 'index': 101},
    {'id': '000519', 'index': 102},
    {'id': '000520', 'index': 103}
]


# 文件路径
qa_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\trend_QA2.json'
data_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\trend_data2.json'
output_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\expresult\trendfs18.json'

# 反义词典（位置敏感）
ANTONYMS = {
  "gradual rise": "fluctuating upward",
  "moderate rise": "fluctuating upward",
  "slow rise": "fluctuating upward",
  "mild uptrend": "fluctuating upward",
  "slight uptrend": "fluctuating upward",
  "volatile but strong": "fluctuating upward",
  "steady rise": "fluctuating upward",
  "incremental rise": "fluctuating upward",
  
  "gradual decline": "fluctuating downward",
  "slow decline": "fluctuating downward",
  "mild downtrend": "fluctuating downward",
  "slight downtrend": "fluctuating downward",
  "volatile but weak": "fluctuating downward",
  "steady decline": "fluctuating downward",
  "incremental decline": "fluctuating downward",
  
  "low volatility": "stable",
  "range-bound": "stable",
  "sideways": "stable",
  "flat": "stable",
  "consolidation": "stable",
  
  "volume-reduced fluctuation": "fluctuation",
  "choppy": "fluctuation",
  "volatile": "fluctuation",
  "mixed": "fluctuation",
  
  "active trading": "volume expansion",
  "brisk trading": "volume expansion",
  "high volume": "volume expansion",
  "strong trading": "volume expansion",
  
  "light trading": "sluggish",
  "thin trading": "sluggish",
  "quiet market": "sluggish",
  "weak trading": "sluggish",
  "inactive": "sluggish",
  "dull": "sluggish",
  
  "neutral": "moderate",
  "moderate risk": "moderate",
  "balanced": "moderate",
  "average": "moderate",
  "fair": "moderate"
}

for slot, pairs in list(ANTONYMS.items()):
    ANTONYMS[slot] |= {(b, a) for (a, b) in pairs}

SYN2CANON = {
    # Upward trend
    "slow rise": "oscillating upward",
    "moderate increase": "oscillating upward",
    "slight increase": "oscillating upward",
    "steady upward movement": "oscillating upward",
    "slightly bullish": "oscillating upward",
    "mildly bullish": "oscillating upward",
    "upward trend": "oscillating upward",

    # Downward trend
    "slow decline": "oscillating downward",
    "slight decrease": "oscillating downward",
    "moderate pullback": "oscillating downward",
    "slightly bearish": "oscillating downward",
    "mildly bearish": "oscillating downward",
    "downward trend": "oscillating downward",

    # Sideways / consolidation
    "low volatility": "stable",
    "sideways movement": "stable",
    "range-bound movement": "oscillating",
    "narrow-range fluctuation": "oscillating",
    "consolidation phase": "oscillating",

    # Volume-related
    "active trading": "high volume",
    "volume expansion": "high volume",
    "high trading volume": "high volume",
    "low-volume fluctuation": "oscillating",
    "thin trading": "inactive",
    "low liquidity": "inactive",
    "shrinking volume": "inactive",

    # Risk / neutral assessment
    "neutral": "normal",
    "moderate risk": "normal",
    "wait-and-see": "normal",
    "high uncertainty": "normal"
}

def normalize_synonym(phrase: str) -> str:
    return SYN2CANON.get(phrase, phrase)

def compute_semantic_score(gt: str, pred: str) -> float:
    P, R, F1 = bert_score([pred], [gt], lang="zh")
    return F1.item()

def extract_float(value):
    match = re.findall(r"\d+\.\d+", value)
    return float(match[0]) if match else None

def is_float_equal(a, b, tol=1e-2):
    try:
        return abs(float(a) - float(b)) < tol
    except:
        return False

# 去除所有"(数字)"槽位标记，并统一逗号
def clean_slots(text: str) -> str:
    text = re.sub(r"\(\d+\)", "", text)
    return text.replace(",", "，")

def is_semantic_correct(gt, pred):
    gt_clean = clean_slots(gt)
    pred_clean = clean_slots(pred)

    gt_parts = [x.strip() for x in gt_clean.split("，") if x.strip()]
    pred_parts = [x.strip() for x in pred_clean.split("，") if x.strip()]

    if len(gt_parts) != len(pred_parts) or len(gt_parts) < 4:
        return False, None, None, None, None

    bert_scores = []
    for idx, (g, p) in enumerate(zip(gt_parts[:4], pred_parts[:4])):
        slot = f"({idx+1})"
        if (g, p) in ANTONYMS.get(slot, set()):
            bert_scores.append(0.0)
            continue
        g_norm = normalize_synonym(g)
        p_norm = normalize_synonym(p)
        if g_norm == p_norm:
            bert_scores.append(1.0)
        else:
            bert_scores.append(compute_semantic_score(g_norm, p_norm))
    text_correct = all(score >= 0.8 for score in bert_scores)

    numerical_scores = []
    mape_sum = rbias_sum = count = 0
    for gt_val, pred_val in zip(gt_parts[4:], pred_parts[4:]):
        gt_num = extract_float(gt_val)
        pred_num = extract_float(pred_val)
        if gt_num is not None and pred_num is not None:
            mape = abs(pred_num - gt_num) / gt_num if gt_num != 0 else 0
            rbias = (pred_num - gt_num) / gt_num if gt_num != 0 else 0
            correct = is_float_equal(gt_num, pred_num)
            mape_sum += mape
            rbias_sum += rbias
            count += 1
            numerical_scores.append({
                "gt_value": gt_val,
                "pred_value": pred_val,
                "correct": correct,
                "mape": round(mape, 6),
                "rbias": round(rbias, 6)
            })
        else:
            numerical_scores.append({
                "gt_value": gt_val,
                "pred_value": pred_val,
                "correct": False,
                "mape": None,
                "rbias": None
            })
    numerical_correct = all(item["correct"] for item in numerical_scores)
    avg_mape = mape_sum / count if count else None
    avg_rbias = rbias_sum / count if count else None

    return text_correct and numerical_correct, bert_scores, numerical_scores, avg_mape, avg_rbias

def is_correct(gt, pred, qtype):
    """
    统一返回 dict
    """
    gt = gt.strip()
    pred = pred.strip()
    result = {
        "correct": False,
        "bert_scores": None,
        "numerical_scores": None,
        "mape": None,
        "rbias": None
    }

    if qtype == "填空题":
        correct, bert_scores, numerical_scores, mape, rbias = is_semantic_correct(gt, pred)
        result.update({
            "correct": correct,
            "bert_scores": bert_scores,
            "numerical_scores": numerical_scores,
            "mape": round(mape, 6) if mape is not None else None,
            "rbias": round(rbias, 6) if rbias is not None else None
        })
    else:
        result["correct"] = (gt == pred)

    return result

# 读取数据
with open(qa_path, 'r', encoding='utf-8-sig') as f:
    qa_data_full = json.load(f)
with open(data_path, 'r', encoding='utf-8-sig') as f:
    stock_data_list = json.load(f)
stock_data = {item['id'].strip(): item['data'] for item in stock_data_list if isinstance(item['id'], str) and 'data' in item}

def format_stock_data(data_list):
    return '\n'.join([
        f"{d['Date']}: 开盘={d['Open']:.2f}, 收盘={d['Close']:.2f}, 最高={d['High']:.2f}, 最低={d['Low']:.2f}, 成交量={d['Volume']}, 涨跌幅={d['Change']}%"
        for d in data_list
    ])

# 准备 few-shot 示例数据
example_data = []
for ex in shot_examples:
    idx = ex['index']
    stock_id = ex['id']
    if idx < 0 or idx >= len(qa_data_full):
        raise IndexError(f"示例索引 {idx} 超出范围，共有 {len(qa_data_full)} 条")
    qa_item = qa_data_full[idx]
    if qa_item['id'] != stock_id:
        raise ValueError(f"索引 {idx} 对应样本ID {qa_item['id']} 与指定ID {stock_id} 不匹配")
    example_data.append(qa_item)

print("\n📋 Few-shot 示例配置：")
for ex in example_data:
    print(f"📘 {ex['question']} (ID={ex['id']})")

# 测试子集
qa_data = qa_data_full[:100]
results = []
MAX_RETRIES = 3
RETRY_DELAY = 7

for item in qa_data:
    stock_id = item['id'].strip()
    qtype = item['type']
    question = item['question']
    answer = item['answer']
    date = item.get('date', '')
    raw_data = stock_data.get(stock_id, [])

    if not raw_data:
        print(f"⚠️ {stock_id} 无历史数据，跳过")
        continue

    print(f"🟡 正在处理 股票ID={stock_id} 的问题：{question}")
    stock_history = format_stock_data(raw_data)

    # 构造 few-shot 示例字符串
    examples_str = ''
    for ex in example_data:
        ex_raw = stock_data.get(ex['id'], [])
        if not ex_raw:
            continue
        ex_history = format_stock_data(ex_raw)
        examples_str += f"股票历史数据：\n{ex_history}\n问题：{ex['question']}\n答案：{ex['answer']}\n\n"

    # 构造 prompt
    full_prompt = f"""
You are a professional financial analyst. Please fill in the blanks below based on the given historical stock data. 
Please note:
(1) If the question is a fill-in-the-blank, only provide the text or number for the blank, without writing full sentences or any extra output;
(2) For calculating resistance and support levels, use the Fibonacci retracement method, and choose the calculation based on the question content;
(3) If it is a true/false question, only answer 'Incorrect'or 'Correct'.

#Here are some examples:
{examples_str}

Stock Price Data:
{stock_history}

Questions:
{question}

Please fill in the answers directly (Chinese punctuation, comma-separated):

"""

    for attempt in range(MAX_RETRIES):
        try:
            response = Application.call(api_key=api_key, app_id=app_id, prompt=full_prompt)
            if response.status_code == HTTPStatus.OK:
                pred = response.output.text.strip()
                result = is_correct(answer, pred, qtype)
                results.append({
                    "id": stock_id,
                    "date": date,
                    "type": qtype,
                    "question": question,
                    "ground_truth": answer,
                    "model_answer": pred,
                    "correct": result["correct"],
                    "bert_scores": result["bert_scores"],
                    "numerical_scores": result["numerical_scores"],
                    "mape": result["mape"],
                    "rbias": result["rbias"]
                })
                break
            else:
                print(f"❌ {stock_id} 请求失败: {response.status_code}")
                break
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"⚠️ {stock_id} 重试中: {e}")
                time.sleep(RETRY_DELAY + random.uniform(0, 2))
            else:
                print(f"❌ {stock_id} 重试失败，跳过")

# 保存结果
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# 汇总统计
correct_count = sum(1 for r in results if r['correct'])
bert_scores_all = [s for r in results if r['bert_scores'] for s in r['bert_scores']]
mape_all = [r['mape'] for r in results if r['mape'] is not None]
rbias_all = [r['rbias'] for r in results if r['rbias'] is not None]
accuracy = correct_count / len(results) if results else 0

print(f"\n✅ 共处理: {len(results)} 条")
print(f"🎯 正确: {correct_count} 条")
print(f"📈 准确率: {accuracy:.2%}")
if bert_scores_all:
    print(f"🧠 文本填空 BERTScore 平均: {sum(bert_scores_all)/len(bert_scores_all):.4f}")
if mape_all:
    print(f"🔢 数值填空 MAPE 平均: {sum(mape_all)/len(mape_all):.4f}")
if rbias_all:
    print(f"🔍 数值填空 RBIAS 平均: {sum(rbias_all)/len(rbias_all):.4f}")


SyntaxError: '{' was never closed (465220131.py, line 42)

In [ ]:
#指标计算——流式
# 3指标计算用MAPE，Rbais衡量误差
# 标记1
# 适用于deepseek-r1、qwen3-235b-a22b、llama-4-scout-17b-16e-instruct、llama-4-maverick-17b-128e-instruct(流式输出)

import os
import json
import time
import random
from http import HTTPStatus
from openai import OpenAI

# ========== 配置参数 ==========
api_key = 'sk-112c402fb4d6405c9518d15c2f5727a7'
base_url = 'https://dashscope.aliyuncs.com/compatible-mode/v1'
client = OpenAI(api_key=api_key, base_url=base_url)

# ========== Few-shot 示例配置：按股票ID和问题编号控制 ==========
# 在这里指定要用作示例的样本，列表中每项包含'id'（股价数据ID）和'index'（qa_data中的索引）


#选择
examples_str = [
    {'id': '605128', 'index': 4431},   
    {'id': '605133', 'index': 4432},  
    {'id': '605136', 'index': 4433}   
]

"""
#基础（填空）
shot_examples = [
    {'id': '601136', 'index': 3634},   
    {'id': '601137', 'index': 3635},  
    {'id': '601138', 'index': 3636}   
]


#困难（填空）
shot_examples = [
    {'id': '000518', 'index': 101},   
    {'id': '000519', 'index': 102},  
    {'id': '000520', 'index': 103}   
]

"""



qa_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\compute_allQA3.json'
data_path = r'D:\PGstudy\studyprojects\pachong\computedatasource\computedata.json'
output_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\expresult\compute_xuanze14.json'

with open(qa_path, 'r', encoding='utf-8') as f:
    full_qa_data = json.load(f)

with open(data_path, 'r', encoding='utf-8-sig') as f:
    stock_data_list = json.load(f)
    stock_data = {
        item['id']: item['data']
        for item in stock_data_list
        if 'id' in item and 'data' in item
    }

# ========== 准备 few-shot 示例数据 ==========
example_data = []
for ex in shot_examples:
    idx = ex['index']
    stock_id = ex['id']
    if idx < 0 or idx >= len(full_qa_data):
        raise IndexError(f"示例索引 {idx} 超出范围，共有 {len(full_qa_data)} 条样本")
    qa_item = full_qa_data[idx]
    if qa_item['id'] != stock_id:
        raise ValueError(f"索引 {idx} 对应样本ID {qa_item['id']} 与指定ID {stock_id} 不匹配")
    example_data.append(qa_item)

print("\n📋 Few-shot 示例配置：")
for ex in example_data:
    print(f"📘 示例问题：{ex['question']}")
    print(f"📈 示例使用的股价数据ID：{ex['id']}")

# ========== 工具函数 ==========
def format_stock_data(data_list):
    return '\n'.join([
        f"{d['Date']}: 开盘={d['Open']:.2f}, 收盘={d['Close']:.2f}, 最高={d['High']:.2f}, 最低={d['Low']:.2f}, 成交量={d['Volume']}, 涨跌幅={d['Change']}%"
        for d in data_list
    ])

def is_correct(gt, pred, qtype):
    gt = gt.strip()
    pred = pred.strip()
    if qtype == "填空题":
        return [x.strip() for x in gt.split('、')] == [x.strip() for x in pred.split('、')]
    return gt == pred

def compute_mape(gt, pred):
    try:
        gt_values = [float(x.strip()) for x in gt.split('、')]
        pred_values = [float(x.strip()) for x in pred.split('、')]
        if len(gt_values) != len(pred_values):
            return None
        mape = sum(abs((p - g) / g) for g, p in zip(gt_values, pred_values) if g != 0) / len(gt_values)
        return round(mape * 100, 4)
    except Exception:
        return None

def compute_rbias(gt, pred):
    try:
        gt_values = [float(x.strip()) for x in gt.split('、')]
        pred_values = [float(x.strip()) for x in pred.split('、')]
        if len(gt_values) != len(pred_values):
            return None
        rbias = sum((p - g) / g for g, p in zip(gt_values, pred_values) if g != 0) / len(gt_values)
        return round(rbias * 100, 4)
    except Exception:
        return None

# ========== 构造few-shot示例prompt ==========
def build_examples_prompt():
    examples_str = ''
    for ex in example_data:
        ex_raw = stock_data.get(ex['id'], [])
        if not ex_raw:
            continue
        ex_history = format_stock_data(ex_raw)
        examples_str += f"股票历史数据：\n{ex_history}\n问题：{ex['question']}\n答案：{ex['answer']}\n\n"
    return examples_str


# ========== 主处理逻辑 ==========
qa_data = full_qa_data[4330:4430]  # 测试用
results = []


for item in qa_data:
    stock_id = item['id']
    qtype = item['type']
    question = item['question']
    answer = item['answer']
    raw_data = stock_data.get(stock_id, [])
    if not raw_data:
        print(f"⚠️ {stock_id} 无历史数据")
        continue

    stock_history = format_stock_data(raw_data)
    print(f"\n🟡 正在处理股票 {stock_id} 的问题...")

    examples_prompt = build_examples_prompt()

    # 构造 prompt
    user_prompt = f"""
 "You are a professional financial analyst. Please fill in the blanks below based on the given historical stock data. 
Please note:
(1) If the question is a fill-in-the-blank, only provide the text or number for the blank, without writing full sentences or any extra output;
(2) For calculating resistance and support levels, use the Fibonacci retracement method, and choose the calculation based on the question content;
(3) If it is a true/false question, only answer 'Incorrect'or 'Correct'.

#Here are some examples:
{examples_str}

Stock Price Data:
{stock_history}

Questions:
{question}

Please fill in the answers directly (Chinese punctuation, comma-separated):"

"""


    try:
        answer_content = ""
        is_answering = False

        completion = client.chat.completions.create(
            model="deepseek-r1",
            #model="qwen3-235b-a22b",
            #model="qwen-turbo-latest",
            #model="deepseek-r1",
            #model="deepseek-v3",
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

        for chunk in completion:
            if not chunk.choices:
                continue
            delta = chunk.choices[0].delta
            if getattr(delta, "content", None):
                answer_content += delta.content

        pred = answer_content.strip()
        correct = is_correct(answer, pred, qtype)

        record = {
            "id": stock_id,
            "type": qtype,
            "question": question,
            "ground_truth": answer,
            "model_answer": pred,
            "correct": correct
        }

        if qtype == "填空题":
            record["mape"] = compute_mape(answer, pred)
            record["rbias"] = compute_rbias(answer, pred)

        results.append(record)

    except Exception as e:
        print(f"🚨 处理 {stock_id} 出错：{e}")
        results.append({
            "id": stock_id,
            "type": qtype,
            "question": question,
            "ground_truth": answer,
            "model_answer": "请求失败",
            "correct": False
        })

# ========== 保存结果 ==========
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# ========== 统计指标 ==========
total_correct = sum(1 for r in results if r['correct'])
accuracy = total_correct / len(results) if results else 0
print(f"\n✅ 共处理: {len(results)} 条，正确: {total_correct} 条，准确率: {accuracy:.2%}")

fill_in_results = [r for r in results if r['type'] == '填空题' and r.get("mape") is not None]
if fill_in_results:
    avg_mape = round(sum(r['mape'] for r in fill_in_results) / len(fill_in_results), 4)
    avg_rbias = round(sum(r['rbias'] for r in fill_in_results if r['rbias'] is not None) / len(fill_in_results), 4)
    fill_correct = sum(1 for r in fill_in_results if r['correct'])
    fill_acc = fill_correct / len(fill_in_results)

    print(f"\n📊 填空题评估结果：")
    print(f" - 样本数: {len(fill_in_results)}")
    print(f" - 准确率: {fill_acc:.2%}")
    print(f" - 平均绝对百分比误差（MAPE）: {avg_mape:.4f}%")
    print(f" - 平均相对偏差（Rbias）: {avg_rbias:.4f}%")
else:
    print("\n⚠️ 没有可用于计算 MAPE/Rbias 的填空题样本")


In [ ]:
#K线型态分析
#标记2
#适用于在阿里云调用的模型
#few-shot版本

import os
import json
import time
import random
from http import HTTPStatus
from dashscope import Application

# 配置 API Key 和 App ID,QWEN-TURBO
api_key = 'sk-112c402fb4d6405c9518d15c2f5727a7'
app_id = 'e832c9f6d9f440658c3a651d6fc49cfa'

# ========== Few-shot 示例配置：按股票ID和问题编号控制 ==========
# 示例列表：每个字典包含 'id'（股票ID）和 'index'（在数据集中的位置）
"""


#第一类
shot_examples = [
    {'id': '000001', 'index': 101},   
    {'id': '000001', 'index': 102},   
    {'id': '002714', 'index': 103}    
]

"""
#第二类
shot_examples = [
    {'id': '002094', 'index': 3001},  
    {'id': '688788', 'index': 3002},   
    {'id': '688788', 'index': 3003}    
]


# 文件路径
qa_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\candlestick_allQA2.json'
data_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\finaldataset\candlestick_data.json'
output_path = r'D:\PGstudy\studyprojects\FinTSQAbenchmark\expresult\candlestick_test_fs8.json'

# 读取问答对
with open(qa_path, 'r', encoding='utf-8') as f:
    full_qa_data = json.load(f)

# 读取股价数据为 dict
with open(data_path, 'r', encoding='utf-8-sig') as f:
    stock_data_list = json.load(f)
    stock_data = {
        item['id']: item['data']
        for item in stock_data_list
        if 'id' in item and 'data' in item
    }

# 格式化股价历史数据
def format_stock_data(data_list):
    return '\n'.join([
        f"{d['Date']}: 开盘={d['Open']:.2f}, 收盘={d['Close']:.2f}, 最高={d['High']:.2f}, 最低={d['Low']:.2f}, 成交量={d['Volume']}, 涨跌幅={d['Change']}%"
        for d in data_list
    ])

# 校验并准备示例数据
example_data = []
for ex in shot_examples:
    idx = ex['index']
    stock_id = ex['id']
    if idx < 0 or idx >= len(full_qa_data):
        raise IndexError(f"示例索引 {idx} 超出范围，共有 {len(full_qa_data)} 条样本")
    qa_item = full_qa_data[idx]
    if qa_item['id'] != stock_id:
        raise ValueError(f"索引 {idx} 对应样本ID {qa_item['id']} 与指定ID {stock_id} 不匹配")
    example_data.append(qa_item)

# ✅ 打印示例信息
print("\n📋 Few-shot 示例配置：")
for ex in example_data:
    print(f"📘 示例问题：{ex['question']}")
    print(f"📈 示例使用的股价数据ID：{ex['id']}")

# 取测试数据
qa_data = full_qa_data[:100]

results = []
MAX_RETRIES = 3
RETRY_DELAY = 5

def is_correct(gt, pred, qtype):
    gt = gt.strip()
    pred = pred.strip()
    if qtype == "填空题":
        return [x.strip() for x in gt.split(',')] == [x.strip() for x in pred.split(',')]
    return gt == pred

# 主循环
for item in qa_data:
    stock_id = item['id']
    qtype = item['type']
    question = item['question']
    answer = item['answer']
    date = item.get('date', '')

    raw_data = stock_data.get(stock_id, [])
    if not raw_data:
        print(f"⚠️ {stock_id} 无历史数据")
        continue

    stock_history = format_stock_data(raw_data)

    # 拼接完整问题
    if 'choices' in item:
        full_question = question.strip() + '\n' + '\n'.join(item['choices'])
    else:
        full_question = question.strip()

    print(f"\n🟡 正在处理问答对中的股票ID: {stock_id}")
    print(f"📋 输入问题（将传入大模型）:\n{full_question}")
    print(f"📈 使用股价数据来自股票ID: {stock_id}")

    # 构造 few-shot 示例字符串
    examples_str = ""
    for ex in example_data:
        ex_raw = stock_data.get(ex['id'], [])
        if not ex_raw:
            continue
        ex_history = format_stock_data(ex_raw)
        if 'choices' in ex:
            ex_question = ex['question'].strip() + '\n' + '\n'.join(ex['choices'])
        else:
            ex_question = ex['question'].strip()
        examples_str += f"股票历史数据：\n{ex_history}\n问题：{ex_question}\n答案：{ex['answer']}\n\n"

    # 构造 prompt（加入 few-shot 示例）
    prompt = f"""   
    You are a financial expert. Please answer 
the following questions based on the historical stock price data provided below. Note:
For multiple-choice questions, select only one from A/B/C/D, without including the word 'Answer.'

# Here are some examples:
{examples_str}

Historical stock data is as follows:
{stock_history}

The questions are as follows:
{full_question}

"""

    # 调用模型
    for attempt in range(MAX_RETRIES):
        try:
            response = Application.call(
                api_key=api_key,
                app_id=app_id,
                prompt=prompt
            )

            if response.status_code == HTTPStatus.OK:
                pred = response.output.text.strip()
                correct = is_correct(answer, pred, qtype)

                results.append({
                    "id": stock_id,
                    "date": date,
                    "type": qtype,
                    "question": full_question,
                    "ground_truth": answer,
                    "model_answer": pred,
                    "correct": correct
                })
                break
            else:
                print(f"{stock_id} 请求失败: {response.status_code}")
                break
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"{stock_id} 错误重试: {str(e)}")
                time.sleep(RETRY_DELAY + random.uniform(0, 2))
            else:
                print(f"{stock_id} 错误跳过")

# 保存结果
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# 输出准确率
accuracy = sum(1 for r in results if r['correct']) / len(results) if results else 0
print(f"\n✅ 共处理: {len(results)} 条，正确: {sum(1 for r in results if r['correct'])} 条，准确率: {accuracy:.2%}")

